In [ ]:
from planet4 import markings, dbscan, io, fnotching, reduction
from pathlib import Path
from dask import dataframe as dd
import pandas as pd
import numpy as np

In [ ]:
io.data_root

Path('/home/ayek72/mnt/slowdata/p4_reduction')

In [ ]:
from nbtools.logging import setup_live_logging
import logging

In [ ]:
setup_live_logging("planet4", logging.WARNING)

<Logger planet4 (WARNING)>

# Data reduction

In [ ]:
fpath = Path("/home/ayek72/Dropbox/data/planet4/panoptes/man_s6.parq")
fpath = Path("/home/ayek72/Dropbox/data/planet4/seasons145.parq")
fpath.exists()

True

In [ ]:
df = pd.read_parquet(fpath)

In [ ]:
df.head()

,classification_id,created_at,image_id,image_url,user_name,marking,x_tile,y_tile,acquisition_date,local_mars_time,...,radius_1,radius_2,distance,angle,spread,version,x_angle,y_angle,image_name,season
9461062,5190322d60f4366429074640,2013-05-13 00:22:05+00:00,APF00014pu,http://www.planetfour.org/subjects/standard/51...,AlexandreGarcia,blotch,2,73,2007-04-03,5:24 PM,...,72.000000,54.000000,NaN,90.000000,NaN,NaN,6.123234e-17,1.000000,PSP_003213_0955,1
9461063,5190322d60f4366429074640,2013-05-13 00:22:05+00:00,APF00014pu,http://www.planetfour.org/subjects/standard/51...,AlexandreGarcia,blotch,2,73,2007-04-03,5:24 PM,...,34.000000,25.500000,NaN,90.000000,NaN,NaN,6.123234e-17,1.000000,PSP_003213_0955,1
9461064,5190322d60f4366429074640,2013-05-13 00:22:05+00:00,APF00014pu,http://www.planetfour.org/subjects/standard/51...,AlexandreGarcia,blotch,2,73,2007-04-03,5:24 PM,...,35.114100,26.335575,NaN,70.016893,NaN,NaN,3.417431e-01,0.939793,PSP_003213_0955,1
9461065,5190322d60f4366429074640,2013-05-13 00:22:05+00:00,APF00014pu,http://www.planetfour.org/subjects/standard/51...,AlexandreGarcia,blotch,2,73,2007-04-03,5:24 PM,...,50.487622,37.865717,NaN,82.030390,NaN,NaN,1.386478e-01,0.990342,PSP_003213_0955,1
9461066,5190322d60f4366429074640,2013-05-13 00:22:05+00:00,APF00014pu,http://www.planetfour.org/subjects/standard/51...,AlexandreGarcia,blotch,2,73,2007-04-03,5:24 PM,...,46.043458,34.532593,NaN,87.510447,NaN,NaN,4.343722e-02,0.999056,PSP_003213_0955,1


In [ ]:
reduction.convert_times(df)

In [ ]:
df.shape

(5288033, 24)

In [ ]:
df.dropna(how='all').shape

(5288033, 24)

In [ ]:
df.dropna(how='all', axis=1).shape

(5288033, 24)

In [ ]:
df = df.dropna(how='all', axis=1)

In [ ]:
df.columns

Index(['classification_id', 'created_at', 'image_id', 'image_url', 'user_name',
       'marking', 'x_tile', 'y_tile', 'acquisition_date', 'local_mars_time',
       'x', 'y', 'image_x', 'image_y', 'radius_1', 'radius_2', 'distance',
       'angle', 'spread', 'version', 'x_angle', 'y_angle', 'image_name',
       'season'],
      dtype='object')

In [ ]:
df.describe()

,classification_id,user_id,workflow_id,workflow_version,subject_ids,len_annot,viewport.width,viewport.height,x,y,tool,frame,distance,spread,rotation,radius_1,radius_2,angle
count,1.024170e+05,1.024170e+05,102417.0,1.024170e+05,1.024170e+05,102417.000000,102417.000000,102417.000000,102417.000000,102417.000000,102417.00000,102417.0,70554.000000,70554.000000,70554.000000,31863.000000,31863.000000,31863.000000
mean,2.200250e+08,1.267740e+06,12978.0,3.230000e+00,4.147373e+07,1860.695588,1449.998887,789.925520,391.943464,308.453877,0.31111,0.0,133.244223,28.844479,78.327512,78.244081,40.081500,-22.552411
std,6.968151e+06,8.982721e+05,0.0,2.424739e-13,1.563920e+04,1308.501232,413.287647,170.924866,238.673550,182.378521,0.46295,0.0,120.937871,21.023718,81.666292,79.664074,40.640963,83.300334
min,2.116558e+08,0.000000e+00,12978.0,3.230000e+00,4.145116e+07,321.000000,256.000000,115.000000,-493.531250,-571.499123,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-270.000000
25%,2.145733e+08,0.000000e+00,12978.0,3.230000e+00,4.145391e+07,924.000000,1280.000000,667.000000,187.968750,155.726303,0.00000,0.0,56.993615,10.000000,22.331447,29.413270,15.244073,-68.198591
50%,2.180138e+08,1.883250e+06,12978.0,3.230000e+00,4.147720e+07,1488.000000,1433.000000,751.000000,386.793732,305.406250,0.00000,0.0,94.107592,24.317378,50.735364,54.411754,27.979509,-26.565051
75%,2.238796e+08,2.013530e+06,12978.0,3.230000e+00,4.148950e+07,2386.000000,1680.000000,916.000000,593.174988,458.976562,1.00000,0.0,164.520425,39.811490,103.240520,98.027700,50.331990,1.051183
max,2.420698e+08,2.076095e+06,12978.0,3.230000e+00,4.148990e+07,11967.000000,4222.000000,2027.000000,1368.906250,963.343079,1.00000,0.0,1435.277159,180.000000,359.999994,1345.925995,672.962997,179.999917


In [ ]:
df.shape

(102417, 42)

In [ ]:
df = reduction.filter_data(df)

In [ ]:
df.shape

(5288033, 24)

In [ ]:
blotches = df.marking == "blotch"

In [ ]:
reduction.convert_ellipse_angles(df)

In [ ]:
reduction.normalize_fan_angles(df)

In [ ]:
df.columns

Index(['classification_id', 'created_at', 'image_id', 'image_url', 'user_name',
       'marking', 'x_tile', 'y_tile', 'acquisition_date', 'local_mars_time',
       'x', 'y', 'image_x', 'image_y', 'radius_1', 'radius_2', 'distance',
       'angle', 'spread', 'version', 'x_angle', 'y_angle', 'image_name',
       'season'],
      dtype='object')

In [ ]:
df.angle.describe()

count    3.802132e+06
mean     1.135086e+02
std      9.541926e+01
min      0.000000e+00
25%      3.756859e+01
50%      8.647855e+01
75%      1.732902e+02
max      3.599415e+02
Name: angle, dtype: float64

In [ ]:
df.rotation.describe()

AttributeError: 'DataFrame' object has no attribute 'rotation'

In [ ]:
df = df.assign(x_angle=np.cos(np.deg2rad(df['angle'])),
               y_angle=np.sin(np.deg2rad(df['angle'])))

In [ ]:
fpath

Path('/home/ayek72/Dropbox/data/planet4/seasons145.parq')

In [ ]:
newfpath = fpath.with_name("man_s6_filtered.parq")
newfpath

Path('/home/ayek72/Dropbox/data/planet4/panoptes/man_s6_filtered.parq')

In [ ]:
df.to_parquet(newfpath)

In [ ]:
ddf = pd.read_parquet(newfpath, filters=[("image_name","==", "ESP_021455_0935")])

In [ ]:
dd.compute(ddf.shape)

In [ ]:
image_names = ddf.image_name.unique()

In [ ]:
img_data = ddf[ddf.image_name=="ESP_021455_0935"]

In [ ]:
img_data.persist()

In [ ]:
data = img_data

In [ ]:
def remove_dupes(data):

    c_ids = []

    def process_user_group(g):
        c_ids.append(g[g.created_at == g.created_at.min()
                       ].classification_id.min())

    data.groupby(['image_id', 'user_name'],
                 sort=False).apply(process_user_group)
    return data.set_index('classification_id').loc[set(c_ids)].reset_index()

# Data to cluster

In [ ]:
from dask.distributed import Client

client = Client()
client

INFO: To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO: State start
INFO:   Scheduler at:     tcp://127.0.0.1:40377
INFO:   dashboard at:            127.0.0.1:8787
INFO:         Start Nanny at: 'tcp://127.0.0.1:42217'
INFO:         Start Nanny at: 'tcp://127.0.0.1:43349'
INFO:         Start Nanny at: 'tcp://127.0.0.1:43899'
INFO:         Start Nanny at: 'tcp://127.0.0.1:37337'
INFO: Register worker <WorkerState 'tcp://127.0.0.1:46715', name: 0, status: init, memory: 0, processing: 0>
INFO: Starting worker compute stream, tcp://127.0.0.1:46715
INFO: Starting established connection to tcp://127.0.0.1:53442
INFO: Register worker <WorkerState 'tcp://127.0.0.1:43573', name: 2, status: init, memory: 0, processing: 0>
INFO: Starting worker compute stream, tcp://127.0.0.1:43573
INFO: Starting established connection to tcp://127.0.0.1:53446
INFO: Register worker <WorkerState 'tcp://127.0.0.1:43955', name: 3, status: 

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 125.52 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40377,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 125.52 GiB
Comm: tcp://127.0.0.1:46715,Total threads: 2
Dashboard: http://127.0.0.1:36137/status,Memory: 31.38 GiB
Nanny: tcp://127.0.0.1:42217,


In [ ]:
from planet4 import region_data, io

In [ ]:
df.created_at.max()

Timestamp('2019-05-02 11:01:23+0000', tz='UTC')

In [ ]:
fpath

Path('/home/ayek72/Dropbox/data/planet4/panoptes/man_s6.parq')

In [ ]:
db = io.DBManager(newfpath)

In [ ]:
any(df.x.isnull())

False

In [ ]:
db.obsids.head()

0    ESP_046642_0935
1    ESP_047696_0935
2    ESP_047327_0935
3    ESP_046971_0935
4    ESP_047103_0935
Name: image_name, dtype: object

# Support functions

In [ ]:
from planet4.catalog_production import (
    ReleaseManager,
    cluster_obsid_parallel,
    fnotch_obsid_parallel,
    fnotch_obsid,
    cluster_obsid
)
from planet4 import fnotching
from planet4.dbscan import DBScanner
from planet4.markings import TileID
from planet4.io import check_and_pad_id, get_subframe, urlretrieve
from planetarypy.utils import url_retrieve
from nbtools import execute_in_parallel
from kalasiris.pysis import ProcessError
from subprocess import CalledProcessError

In [ ]:
rm = ReleaseManager("v3.0_Manhattan", dbname=db.dbname, overwrite=False)

In [ ]:
rm.catalog

'P4_catalog_v3.0_Manhattan'

In [ ]:
rm.check_for_todo()

In [ ]:
rm.todo

['ESP_046642_0935',
 'ESP_047696_0935',
 'ESP_047327_0935',
 'ESP_046971_0935',
 'ESP_047103_0935',
 'ESP_047459_0935',
 'ESP_046681_0935',
 'ESP_047961_0935']

## clustering

In [ ]:
obsid = rm.todo[0]
obsid

'ESP_046642_0935'

In [ ]:
savedir = Path("/home/ayek72/mnt/slowdata/p4_reduction")

In [ ]:
df.columns

Index(['classification_id', 'user_name', 'user_id', 'user_ip', 'workflow_id',
       'workflow_name', 'workflow_version', 'created_at', 'subject_ids',
       'len_annot', 'source', 'session', 'started_at', 'user_agent',
       'utc_offset', 'finished_at', 'live_project', 'user_language',
       'user_group_ids', 'workflow_translation_id', 'viewport.width',
       'viewport.height', 'subject_selection_state.retired',
       'subject_selection_state.selected_at',
       'subject_selection_state.already_seen',
       'subject_selection_state.selection_state',
       'subject_selection_state.finished_workflow',
       'subject_selection_state.user_has_finished_workflow', 'seen_before',
       'image_id', 'image_name', 'x', 'y', 'tool', 'frame', 'distance',
       'spread', 'rotation', 'marking', 'radius_1', 'radius_2', 'angle',
       'x_angle', 'y_angle'],
      dtype='object')

In [ ]:
cluster_obsid(obsid, savedir, dbname=db.dbname)

  0%|          | 0/174 [00:00<?, ?it/s]

INFO: full garbage collection released 44.89 MiB from 2073 reference cycles (threshold: 9.54 MiB)


ValueError: Input X contains NaN.
DBSCAN does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
rm.catalog

'P4_catalog_v2.0_Manhattan'

In [ ]:
rm.dbname

Path('/home/maye/big_drive/maye/local_data/planet4/manhattan_queryable_cleaned.parq')

In [ ]:
rm.calc_tile_coordinates()

100%|██████████| 73/73 [00:01<00:00, 72.05it/s]
INFO: Wrote /home/maye/big_drive/processed_data/planet4/catalogs/P4_catalog_v2.0_Manhattan/P4_catalog_v2.0_Manhattan_tile_coords.csv


In [ ]:
rm.calc_marking_coordinates()

In [ ]:
rm.calc_metadata()

AttributeError: 'DBManager' object has no attribute 'get_all'

In [ ]:
db.image_ids??

Type:        property
String form: <property object>
Source:     
# db.image_ids.fget
@property
def image_ids(self):
    "Return list of unique image_ids in database."
    return self.df.image_id.unique().compute()


In [ ]:
rm.launch_catalog_production()

In [ ]:
from planet4.projection import create_RED45_mosaic, nocal_hi, get_RED45_mosaic_inputs

In [ ]:
from kalasiris import spiceinit

In [ ]:
from tqdm.auto import tqdm

In [ ]:
overwrite=True

def do_first_part(obsid):
    products = get_RED45_mosaic_inputs(obsid)
    mos_path = products[0].local_path.parent / f"{obsid}_mosaic_RED45.cub"

    # bail out if exists:
    if mos_path.exists() and not overwrite:
        print(f"{mos_path} already exists and I am not allowed to overwrite.")
        return obsid, True

    for prod in products:
        prod.download(overwrite=overwrite)
        nocal_hi(prod)

In [ ]:
for obsid in tqdm(rm.obsids):
    try:
        do_first_part(obsid)
    except CalledProcessError as e:
        print(e.stdout)
        print(e.stderr)

In [ ]:
from dask import delayed, compute

In [ ]:
lazys = []
for obsid in rm.obsids:
    lazys.append(delayed(create_RED45_mosaic)(obsid, overwrite=True))

In [ ]:
results = compute(*lazys)

### Metadata

In [ ]:
from planet4.metadata import MetadataReader

In [ ]:
md = MetadataReader(rm.todo[0])

In [ ]:
md.proj_folder

Path('/home/maye/big_drive/processed_data/planet4/ground_projection')

In [ ]:
from planet4.io import get_ground_projection_root, get_config

In [ ]:
from planetarypy.config import config

In [ ]:
config.storage_root

In [ ]:
get_ground_projection_root()

In [ ]:
rm.calc_metadata()

In [ ]:
%debug

In [ ]:
rm.merge_all()

In [ ]:
blotches = pd.read_csv(rm.blotch_merged)

In [ ]:
blotches.head()

In [ ]:
pd.read_csv(rm.fan_merged).shape

In [ ]:
rm.fan_merged

In [ ]:
fans = pd.read_csv(self.fan_file)
blotches = pd.read_csv(self.blotch_file)
meta = pd.read_csv(self.metadata_path, dtype='str')
tile_coords = pd.read_csv(self.tile_coords_path, dtype='str')

In [ ]:
meta.shape

In [ ]:
tile_coords.shape

In [ ]:
cols_to_merge = ['OBSERVATION_ID',
                         'SOLAR_LONGITUDE', 'north_azimuth', 'map_scale']
fans = fans.merge(meta[cols_to_merge],
                  left_on='obsid', right_on='OBSERVATION_ID')
blotches = blotches.merge(
    meta[cols_to_merge], left_on='obsid', right_on='OBSERVATION_ID')

In [ ]:
fans.shape

In [ ]:
blotches.shape

In [ ]:
fans.drop(self.DROP_FOR_FANS, axis=1, inplace=True)
blotches.drop(self.DROP_FOR_BLOTCHES, axis=1, inplace=True)

In [ ]:
blotches.shape

In [ ]:
fans, blotches = self.merge_fnotch_results(fans, blotches)

In [ ]:
fans.shape

In [ ]:
blotches.shape

In [ ]:
from planet4 import catalog_production as cp

In [ ]:
rm.EDRINDEX_meta_path

In [ ]:
rm.merge_all()

In [ ]:
rm.calc_metadata()

In [ ]:
meta = pd.read_csv(rm.metadata_path).set_index('OBSERVATION_ID')

In [ ]:
rm.EDRINDEX_meta_path

In [ ]:
meta.loc['ESP_012251_0935']

In [ ]:
edrindex = pd.read_hdf("/Volumes/Data/hirise/EDRCUMINDEX.hdf")
p4_edr = edrindex[edrindex.OBSERVATION_ID.isin(rm.obsids)].query(
    'CCD_NAME=="RED4"').drop_duplicates(subset='OBSERVATION_ID')

In [ ]:
p4_edr.set_index('OBSERVATION_ID', inplace=True)

In [ ]:
p4_edr.loc['ESP_012251_0935']

In [ ]:
rm.calc_metadata()

In [ ]:
rm.

In [ ]:
df.set_index('OBSERVATION_ID').to_csv(rm.EDRINDEX_meta_path, index=True)

In [ ]:
cols = ['OBSERVATION_ID', 'IMAGE_CENTER_LATITUDE', 'IMAGE_CENTER_LONGITUDE', 'SOLAR_LONGITUDE', 'START_TIME',
        'north_azimuth', '# of tiles']

In [ ]:
df[cols].head()

In [ ]:
fname = "P4_catalog_v1.0_EDRINDEX_metadata.csv"
df.to_csv(rm.savefolder / fname, index=False)

In [ ]:
rm.savefolder

In [ ]:
import itertools
import string 

# Adapt this prefix as a unique identifier of the dataset

def fan_id_generator():
    for newid in itertools.product(string.digits+'abcdef', repeat=6):
        yield 'F' + ''.join(newid)

def blotch_id_generator():
    for newid in itertools.product(string.digits+'abcdef', repeat=6):
        yield 'B' + ''.join(newid)

In [ ]:
def add_marking_ids(path, fan_id, blotch_id):
    """Add marking_ids for catalog to cluster results.
    
    Parameters
    ----------
    path : str, pathlib.Path
        Path to L1A image_id clustering result directory
    fan_id, blotch_id : generator
        Generator for marking_id
    """
    image_id = path.parent.name
    for kind, id_ in zip(['fans', 'blotches'], [fan_id, blotch_id]):
        fname = str(path / f"{image_id}_L1A_{kind}.csv")
        try:
            df = pd.read_csv(fname)
        except FileNotFoundError:
            continue
        else:
            marking_ids = []
            for i in range(df.shape[0]):
                marking_ids.append(next(id_))
            df['marking_id'] = marking_ids
            df.to_csv(fname, index=False)

In [ ]:
from tqdm import tqdm

In [ ]:
obsids = rm.obsids

In [ ]:
rm.launch_catalog_production()

In [ ]:
rm.calc_metadata()

In [ ]:
rm.calc_marking_coordinates()

In [ ]:
rm.merge_all()

In [ ]:
fan_id = fan_id_generator()
blotch_id = blotch_id_generator()

In [ ]:
from planet4.fnotching import fnotch_image_ids, apply_cut

In [ ]:
for obsid in tqdm(obsids):
    pm = io.PathManager(obsid=obsid, datapath=rm.savefolder)
    paths = pm.get_obsid_paths('L1A')
    for path in paths:
        add_marking_ids(path, fan_id, blotch_id)

In [ ]:
rm.catalog

In [ ]:
for obsid in tqdm(obsids):
    fnotch_image_ids(obsid, savedir=rm.catalog)
    apply_cut(obsid, savedir=rm.catalog)

In [ ]:
from planet4.catalog_production import create_roi_file

In [ ]:
create_roi_file(obsids, rm.catalog, rm.catalog)

In [ ]:
rm.get_metadata()

In [ ]:
rm.merge_all()

In [ ]:
fans = rm.read_fan_file()

In [ ]:
fans.columns

In [ ]:
blotches = rm.read_blotch_file()

In [ ]:
blotches.columns

In [ ]:
pd.read_csv(rm.metadata_path).columns

In [ ]:
fans.vote_ratio.isna().value_counts()

In [ ]:
fans.head()

In [ ]:
rm.launch_catalog_production()

# Debugging

In [ ]:
from planet4.fnotching import get_clusters_in_path

def find_bad_obsid(obsid, savedir):
    pm = io.PathManager(obsid=obsid, datapath=savedir)
    paths = pm.get_obsid_paths('L1A')
    if len(paths) == 0:
        logger.warning("No paths to fnotch found for %s", obsid)
    fans = []
    blotches = []
    for path in paths:
        f, b = get_clusters_in_path(path)
        fans.append(f)
        blotches.append(b)
    fans = pd.concat(fans, ignore_index=True)

In [ ]:
folder

In [ ]:
for obsid in obsids:
    print(obsid)
    find_bad_obsid(obsid, folder)

In [ ]:
import holoviews as hv
hv.notebook_extension()

In [ ]:
from planet4.io import get_subframe

In [ ]:
obsid = 'ESP_022273_0950'

In [ ]:
data = db.get_image_name_markings(obsid)[['image_id', 'image_url']]

In [ ]:
data.set_index('image_id', drop=True, inplace=True)

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
def show_subframe(image_id):
    img = get_subframe(data.loc[data.index[0]].values[0])
    return hv.Raster(img)

In [ ]:
%output size=150

In [ ]:
%%opts Raster [xaxis=None, yaxis=None]
show_subframe(data.index[0])+show_subframe(data.index[1])

In [ ]:
dmap = hv.DynamicMap(show_subframe, kdims=['image_id'])
dmap

In [ ]:
dmap.redim.values(image_id=data.index.values)

In [ ]:
xvals = np.linspace(-4,0,202)
yvals = np.linspace(4,0,202)
xs,ys = np.meshgrid(xvals, yvals)

def waves_image(alpha, beta):
    return hv.Image(np.sin(((ys/alpha)**alpha+beta)*xs))

waves_image(0,0) + waves_image(0,4)

In [ ]:
dmap = hv.DynamicMap(waves_image, kdims=['alpha', 'beta'])
dmap

In [ ]:
dmap[0,1] + dmap.select(alpha=1, beta=2)

In [ ]:
dmap.redim.range(alpha=(0,5.0), beta=(1,5.0))

In [ ]:
hv.__version__

In [ ]:
for imgid in image_ids:
    do_cluster_ids(imgid, "without_large_fan_fnotch_on_p4_coords" )

In [ ]:
obsid = 'ESP_020930_0980'
savedir = 'fnotching_per_obsid'

In [ ]:
dbscanner = dbscan.DBScanner(savedir=savedir)

In [ ]:
dbscanner.cluster_image_name(obsid)

In [ ]:
pm.final_blotchfile

In [ ]:
fnotching.fnotch_obsid(obsid, savedir=savedir)

In [ ]:
fnotching.apply_cut_obsid(obsid, savedir=savedir)

In [ ]:
data = region_data.Inca.season2 + region_data.Inca.season3

In [ ]:
do_clustering(ithaca[0])

In [ ]:
ithaca[0]

In [ ]:
from nbtools import execute_in_parallel

In [ ]:
result = execute_in_parallel(do_clustering, data)

# Creating obsid files

In [ ]:
from planet4.fnotching import get_clusters_in_path

In [ ]:
pm = io.PathManager(obsid=roi.seasons[0], datapath=folder)

In [ ]:
pm.obsid_final_blotches_path

In [ ]:
from planet4.catalog_production import create_roi_file

In [ ]:
savedir

In [ ]:
create_roi_file(obsids, 'catalog', savedir)

In [ ]:
pm = io.PathManager(obsid=obsids[0], datapath=savedir)

In [ ]:
folders = pm.get_obsid_paths('L1C')

In [ ]:
from planet4.catalog_production import read_csvfiles_into_lists_of_frames, concat_bucket_lists

In [ ]:
bucket.items()

In [ ]:
roi.available_seasons

In [ ]:
folder

In [ ]:
def create_roi_file_with_seasons(roi, datapath, copy_to_analysis=True):
    Fans = []
    Blotches = []
    for season in roi.available_seasons:
        print(season)
        # keep lower case marking objects to add season column later
        fans = []
        blotches = []
        for obsid in getattr(roi, season):
            print(obsid)
            pm = io.PathManager(obsid=obsid, datapath=datapath)
            fans.append(pm.final_fandf)
            blotches.append(pm.final_blotchdf)

        fans = pd.concat(fans, ignore_index=True)
        fans['season'] = season
        Fans.append(fans)
    
        blotches = pd.concat(blotches, ignore_index=True)
        blotches['season'] = season
        Blotches.append(blotches)
        
    Fans = pd.concat(Fans, ignore_index=True)
    Blotches = pd.concat(Blotches, ignore_index=True)
    savedir = pm.path_so_far.parent
    for marks in ['Fans', 'Blotches']:
        savename = f"{roi.name}_{pm.L1C_folder}_{marks}.csv"
        savepath = savedir / savename
        obj = eval(marks)  # get handle to the fan or blotch object
        obj = obj.dropna(how='all', axis=1)
        obj.to_csv(savepath, index=False)
        if copy_to_analysis is True:
            savefolder = io.analysis_folder() / folder / f"{roi.name.lower()}"
            savefolder.mkdir(exist_ok=True)
            savepath = savefolder / savename
            obj.to_csv(savepath, index=False)

In [ ]:
create_roi_file_with_seasons(roi, folder)

In [ ]:
def create_obsid_result_files(obsid, datapath=None):
#     logger.debug("Working on %s", obsid)
    pm = io.PathManager(obsid=obsid, datapath=datapath)
    # the pm.get_obsid_paths method searches for existing image_id folders inside each
    # obsid folder
    folders = list(pm.get_obsid_paths(pm.L1C_folder))
    blotches = []
    fans = []
    for folder in folders:
        pm.id = fnotching.get_id_from_path(folder)
        if pm.final_blotchfile.exists() and pm.final_blotchfile.lstat().st_size > 1:
            blotches.append(pm.final_blotchdf)
        if pm.final_fanfile.exists() and pm.final_fanfile.lstat().st_size > 1:
            fans.append(pm.final_fandf)

    if len(blotches) > 0:
        blotches = pd.concat(blotches, ignore_index=True)
        blotches.dropna(how='all', axis=1, inplace=True)
        blotches.to_csv(pm.obsid_final_blotches_path, index=False)
        print("Created", pm.obsid_final_blotches_path)

    if len(fans) > 0:
        fans = pd.concat(fans, ignore_index=True)
        fans.dropna(how='all', axis=1, inplace=True)
        fans.to_csv(pm.obsid_final_fans_path, index=False)
        print("Created", pm.obsid_final_fans_path)

In [ ]:
for obsid in data:
    create_obsid_result_files(obsid, datapath='for_chase')

In [ ]:
region = getattr(region_data, name)
for season_no in [2, 3]:
    season = str(season_no)
    bucket = []
    for obsid in getattr(region, f"season{season}"):
        pm = io.PathManager(obsid=obsid)
        df = pd.read_csv(pm.obsid_final_blotches_path)
        df['obsid'] = obsid
        bucket.append(df)

    df = pd.concat(bucket, ignore_index=True)
    df.to_csv(io.analysis_folder() / f"p4_catalog/{name}_season{season}_blotches.csv",
              index=False)

In [ ]:
store_folder = io.analysis_folder() / 'for_chase'
store_folder.mkdir(exist_ok=True)

In [ ]:
name = 'inca'

In [ ]:
p = Path('/Users/klay6683/Dropbox/data/planet4/for_chase/' + name)
for kind in ['blotches', 'fans']:
    bucket = []
    filepaths = p.glob(f'*_{kind}.csv')
    for filepath in filepaths:
        obsid = filepath.name[:15]
        df = pd.read_csv(filepath)
        df['obsid'] = obsid
        bucket.append(df)
    df = pd.concat(bucket, ignore_index=True)
    savedir = store_folder / f'{name}'
    savedir.mkdir(exist_ok=True)
    savepath = savedir / f"{name}_{kind}.csv"
    print(savepath)
    df.to_csv(savepath, index=False)
        